In [2]:
import sys
sys.path.append("/Users/ashadi/study assistant/studdybuddy")

In [3]:
from pydantic import BaseModel
from typing import Optional


from agents.goalBreakdown_agent.agent import root_agent as agent

In [18]:
goal_data = {
 'subject': 'Mathematics',
 'title': 'Prepare for the final exam in Calculus',
 'deadline': '2024-06-30T23:59:59',
 'description': '',
 }
import json
from backend.app.core.models.goal import Goal

goal_instance = Goal(**goal_data).model_dump()
goal_instance['education_level'] = 'university'
goal_instance['age'] = 17
goal_str = json.dumps(goal_instance)
type(goal_str)


str

In [19]:
goal_instance

{'subject': 'Mathematics',
 'title': 'Prepare for the final exam in Calculus',
 'deadline': '2024-06-30T23:59:59',
 'description': '',
 'education_level': 'university',
 'age': 17}

In [20]:
import uuid
from dotenv import load_dotenv
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
from agents.goalBreakdown_agent import root_agent

In [21]:
load_dotenv()

True

In [22]:
session_service_stateful = InMemorySessionService()
initial_state={
    "objectivesAndRequirements":"",
    "plan":"",
    "review":""
}

In [23]:

APP_NAME="StuddyBuddy"
USER_ID='user1'
SESSION_ID=str(uuid.uuid4())

In [24]:
runner = Runner(
    agent=root_agent,
    app_name=APP_NAME,
    session_service=session_service_stateful,
)

In [25]:
content = types.Content(
    role="user", parts=[types.Part(text=goal_str)]
)

type(content)

google.genai.types.Content

In [26]:
existing_sessions = await session_service_stateful.list_sessions(
        app_name=APP_NAME,
        user_id=USER_ID,
    )

    # If there's an existing session, use it, otherwise create a new one
if existing_sessions and len(existing_sessions.sessions) > 0:
        # Use the most recent session
        SESSION_ID = existing_sessions.sessions[0].id
        print(f"Continuing existing session: {SESSION_ID}")
else:
        # Create a new session with initial state
        new_session = await session_service_stateful.create_session(
            app_name=APP_NAME,
            user_id=USER_ID,
            state=initial_state,
        )
        SESSION_ID = new_session.id
        print(f"Created new session: {SESSION_ID}")

Created new session: b87c5d6f-1aed-4a5b-8f27-b7c093278fff


In [27]:

async for event in runner.run_async(
    user_id=USER_ID, session_id=SESSION_ID, new_message=content
):
    if event.is_final_response():
        if(event.content
            and event.content.parts
            and hasattr(event.content.parts[0], "text")
            and event.content.parts[0].text
        ):
            final_response = event.content.parts[0].text.strip()
            print(final_response)





## Requirements and Objectives Analysis for Calculus Final Exam Preparation

Based on the provided input, here's an analysis of the requirements and objectives for preparing for the Calculus final exam:

**I. Requirements:**

*   **Requirement 1: Comprehensive Content Coverage**
    *   **Rationale:**  The user needs to cover all topics that will be tested on the final exam. This is the foundational requirement for success.
    *   **Priority:** High
*   **Requirement 2: Problem-Solving Proficiency**
    *   **Rationale:** Calculus is a problem-solving-based subject.  The user needs to be able to apply concepts to solve a variety of problems.
    *   **Priority:** High
*   **Requirement 3: Exam-Specific Strategies**
    *   **Rationale:** Understanding the exam format, types of questions, and time management strategies is crucial for performing well under pressure.
    *   **Priority:** Medium
*   **Requirement 4: Error Analysis and Correction**
    *   **Rationale:** Identifying and c


------EXIT LOOP TRIGGERED------
Plan review completed successfully
Loop will exit now
---------------------------------



In [28]:
session = await session_service_stateful.get_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
plan = session.state.get("plan","unknown")

In [29]:
plan

'```json\n{\n  "Total time to complete the roadmap": "62 hours",\n  "Level of difficulty in completing the roadmap": "Medium",\n  "Number of total subtasks": 12,\n  "Subtasks": [\n    {\n      "Task Title": "Subtask 1: Gather Resources",\n      "Priority level": "High",\n      "Task description": "Gather all necessary resources (textbooks, notes, past exams, online resources).",\n      "Time duration": "2 hours",\n      "When is it due respect to final deadline": "2024-05-02 23:59:59",\n      "Pro Tips": "Organize resources by topic for easy access."\n    },\n    {\n      "Task Title": "Subtask 2: Understand Exam Format",\n      "Priority level": "Medium",\n      "Task description": "Review the exam syllabus and past papers to identify common question types, topics, and time allocation for each section.",\n      "Time duration": "2 hours",\n      "When is it due respect to final deadline": "2024-05-04 23:59:59",\n      "Pro Tips": "Pay attention to the weighting of different topics. No

In [30]:
import re
match = re.search(r"\{.*\}", plan, re.DOTALL)
if match:
    plan_json_str = match.group(0)
    plan_obj = json.loads(plan_json_str)
    print(plan_obj)
    print(type(plan_obj))
else:
    print("No JSON found in string.")

{'Total time to complete the roadmap': '62 hours', 'Level of difficulty in completing the roadmap': 'Medium', 'Number of total subtasks': 12, 'Subtasks': [{'Task Title': 'Subtask 1: Gather Resources', 'Priority level': 'High', 'Task description': 'Gather all necessary resources (textbooks, notes, past exams, online resources).', 'Time duration': '2 hours', 'When is it due respect to final deadline': '2024-05-02 23:59:59', 'Pro Tips': 'Organize resources by topic for easy access.'}, {'Task Title': 'Subtask 2: Understand Exam Format', 'Priority level': 'Medium', 'Task description': 'Review the exam syllabus and past papers to identify common question types, topics, and time allocation for each section.', 'Time duration': '2 hours', 'When is it due respect to final deadline': '2024-05-04 23:59:59', 'Pro Tips': 'Pay attention to the weighting of different topics. Note any specific instructions or formulas provided.'}, {'Task Title': 'Subtask 3: Review Limits and Continuity', 'Priority leve

In [1]:
import sys
sys.path.append("/Users/ashadi/study assistant/studdybuddy")

In [2]:
from agents.goalBreakdown_agent import root_agent
from backend.app.core.services.agentCall_service import call_agent
import json
input = {
  "age": 17,
  "education_level": "High School",
  "subject": "Biology",
  "title": "Prepare for AP Biology Final Exam",
  "deadline": "2024-07-15T23:59:59Z",
  "Current date": "2024-06-30T12:00:00Z",
  "description": "I want to score at least a 4 on the AP Biology exam. I struggle most with genetics and cell biology. I have extracurricular activities on weekdays after 5pm, so I can only study in the evenings and on weekends. I prefer visual learning resources and practice tests."
}
input = json.dumps(input)

response = await call_agent(root_agent,'plan',{},'user2',input)



Created new session: bcfd56c9-e03f-4306-9b4e-96f0bbc13af8
Okay, I will analyze the user's input to extract and define the requirements and objectives for preparing for the AP Biology final exam.

**Requirements:**

*   **Requirement 1: Achieve a score of 4 or higher on the AP Biology exam.**
    *   Rationale: This is the explicitly stated target score and defines the level of success.
    *   Priority: HIGH (Critical for success)
*   **Requirement 2: Focus on Genetics and Cell Biology.**
    *   Rationale: The user has identified these areas as weaknesses requiring more attention. Addressing these weaknesses is crucial for improving overall performance.
    *   Priority: HIGH (Critical for success)
*   **Requirement 3: Utilize visual learning resources.**
    *   Rationale: The user expresses a preference for visual learning, indicating that this method is more effective for their comprehension and retention.
    *   Priority: MEDIUM (Important for efficient learning)
*   **Requiremen


------EXIT LOOP TRIGGERED------
Plan review completed successfully
Loop will exit now
---------------------------------

{'estimatedTime': 25, 'difficulty': 'high', 'Number of total subtasks': 7, 'subtasks': [{'title': 'Prepare for AP Biology Final Exam - Subtask 1: Diagnostic Assessment', 'priority': 'high', 'instructions': 'Take a full-length AP Biology practice test to identify your strengths and weaknesses. Focus on understanding the reasoning behind each answer, not just memorizing facts.', 'execises': ['Download a full-length AP Biology practice exam from a reputable source (e.g., College Board, Kaplan, Princeton Review).', 'Simulate exam conditions: time yourself, minimize distractions, and avoid using notes or textbooks.', 'Score your exam and analyze your performance. Identify areas where you struggled, particularly in genetics and cell biology.'], 'estimatedTime': 3, 'dueDate': '2024-07-01', 'tips': 'Review the scoring guidelines to understand how points are awarded. Pay att

In [3]:
response

{'estimatedTime': 25,
 'difficulty': 'high',
 'Number of total subtasks': 7,
 'subtasks': [{'title': 'Prepare for AP Biology Final Exam - Subtask 1: Diagnostic Assessment',
   'priority': 'high',
   'instructions': 'Take a full-length AP Biology practice test to identify your strengths and weaknesses. Focus on understanding the reasoning behind each answer, not just memorizing facts.',
   'execises': ['Download a full-length AP Biology practice exam from a reputable source (e.g., College Board, Kaplan, Princeton Review).',
    'Simulate exam conditions: time yourself, minimize distractions, and avoid using notes or textbooks.',
    'Score your exam and analyze your performance. Identify areas where you struggled, particularly in genetics and cell biology.'],
   'estimatedTime': 3,
   'dueDate': '2024-07-01',
   'tips': 'Review the scoring guidelines to understand how points are awarded. Pay attention to the format of free-response questions.'},
  {'title': 'Prepare for AP Biology Final